In [ ]:
pip install anvil-uplink

In [ ]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving medium_data.csv.zip to medium_data.csv (1).zip


In [ ]:
text_df = pd.read_csv("medium_data.csv.zip")
text = list(text_df.title.values)
joined_text = " ".join(text)

with open("joined_text.txt", "w", encoding="utf-8") as f:
    f.write(joined_text)

In [ ]:
import anvil.server

anvil.server.connect("server_F32JSH6ZGZ2BW2T7L4TZS7P6-EPQKLGEWOQYLAQFS")

In [ ]:
partial_text = joined_text[:100000]

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [ ]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [ ]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [ ]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)

In [ ]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [ ]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X, y, batch_size=128, epochs=10, shuffle=True).history

Epoch 1/10
129/129 [==============================] - 29s 197ms/step - loss: 6.9730 - accuracy: 0.0318
Epoch 2/10
129/129 [==============================] - 25s 194ms/step - loss: 6.6024 - accuracy: 0.0636
Epoch 3/10
129/129 [==============================] - 25s 193ms/step - loss: 6.3603 - accuracy: 0.0932
Epoch 4/10
129/129 [==============================] - 26s 198ms/step - loss: 6.1002 - accuracy: 0.1114
Epoch 5/10
129/129 [==============================] - 25s 197ms/step - loss: 5.8279 - accuracy: 0.1298
Epoch 6/10
129/129 [==============================] - 26s 200ms/step - loss: 5.5439 - accuracy: 0.1478
Epoch 7/10
129/129 [==============================] - 25s 194ms/step - loss: 5.2512 - accuracy: 0.1705
Epoch 8/10
129/129 [==============================] - 25s 195ms/step - loss: 4.9441 - accuracy: 0.1982
Epoch 9/10
129/129 [==============================] - 25s 196ms/step - loss: 4.6340 - accuracy: 0.2302
Epoch 10/10
129/129 [==============================] - 25s 196ms/step - l

In [ ]:
model.save("mymodel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model=load_model("mymodel.h5")

In [ ]:
def predict_next_word(input_text,n_best):
  n_best=int(n_best)
  input_text=input_text.lower()
  X=np.zeros((1,n_words,len(unique_tokens)))
  for i,word in enumerate(input_text.split()):
    X[0,i,unique_token_index[word]]=1
  predictions=model.predict(X)[0]
  return (np.argpartition(predictions,-n_best)[-n_best:])

In [ ]:
@anvil.server.callable
def predict_next_word1(input_text,n_best):
  possible=predict_next_word(input_text,n_best)
  return ([unique_tokens[idx] for idx in possible])


In [ ]:
anvil.server.connect("server_F32JSH6ZGZ2BW2T7L4TZS7P6-EPQKLGEWOQYLAQFS")
anvil.server.wait_forever()

1/1 [==============================] - 0s 32ms/step
